In [6]:
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import pickle



# Get the list of recordings
info = mfun.analysis()
# display the detected session
print( "Total Session: " +  str(info.recordingList .shape[0]))
#info.recordingList.head()

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')

# Filter for a particular expRef from the info list (Antara)
specific_expRef = '2023-05-31_1_OFZ011'

# Recreate info object with the specific expRef
info.recordingList = info.recordingList[info.recordingList['sessionName'] == specific_expRef].reset_index(drop=True)


Takes <30s. However, it might take >1 mins or so if network is busy...
Computer: Candela Windows


c:\Users\Lak Lab\Documents\Github\sideBiasLateralisation\main_funcs.py:148: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2023-05-31_OFZ011_1\' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.recordingList.loc[ind,'analysispathname'] = analysispathname +'\\'


Total Session: 58
Matlab engine is set correctly.


In [20]:
info.recordingList


,animalID,recordingDate,recordingID,sessionName,learningData,twoP,ROI,path,sessionNameWithPath,analysispathname,filepathname,eventTimesExtracted
0,OFZ011,2023-05-31,1,2023-05-31_1_OFZ011,True,True,0,Z:/OFZ011\2023-05-31,Z:/OFZ011\2023-05-31\1\2023-05-31_1_OFZ011_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/OFZ011\2023-05-31\1,0.0


In [ ]:
# # to check the paq file details - change filenamePAQ
# import paq2py as paq_reader
# import utils_funcs as utils

# filenamePAQ =  'Y:\\OFZ011\\2023-07-07\\TwoP\\2023-07-07_OFZ011_paq_001.paq'
# savepathname = info.recordingList()
# paq_data = paq_reader.paq_read( file_path=filenamePAQ, plot=True, save_path=savepathname) 
# frame_clock = utils.paq_data (paqData, 'reward', threshold_ttl=True, plot=False)

In [ ]:
# Check CSV files 
# checkOnly needs to be False to create new files with MATLAB GENERIC CODE: GetBehavData.mat
checkOnly = False # Make false when there is more behaviour session
alignSubtract = True
#ind = 49
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
#if ind ==49:
    filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
    e_filenameCSV = [f for f in glob.glob(filenameCSV)]
    if len(e_filenameCSV)==1:
        info.recordingList.loc[ind,'eventTimesExtracted']=1
        info.recordingList.loc[ind,'eventTimesPath'] = filenameCSV
    else:
        if checkOnly:
            info.recordingList.loc[ind,'eventTimesExtracted']=0
            info.recordingList.loc[ind,'eventTimesPath'] = filenameCSV
        else:
            try:
                filenameTimeline = [f for f in glob.glob(info.recordingList.filepathname[ind]+ '\\' + info.recordingList.sessionName[ind] + '_Timeline.mat')]
                if  (len(filenameTimeline)>0):
                    sessionProfile ='Value2AFC'
                else:
                    sessionProfile ='Value2AFC_noTimeline'
                
                # Get behaviour trial data from Block.mat file
                print(' Extracting time events--> Profile: ' + sessionProfile  +'  Session: ' + info.recordingList.sessionName[ind])
                data = eng.getBehavData(info.recordingList.sessionName[ind],sessionProfile)

                # Apply correction based on the weights to match the eventTimes
                if (alignSubtract) & (len(filenameTimeline)>0) & (info.recordingList.twoP[ind]==True): 
                    print(' Aligning time events: ' + info.recordingList.sessionName[ind])
                    # Get weights to convert from probe to behavioural timebase
                    twoPpath = info.recordingList.path[ind] + '\\TwoP'
                    sessionName = info.recordingList.sessionName[ind]
                    figsavepath = info.recordingList.analysispathname[ind]
                    dataCorrected, variance = eng.applySubtractionCorrection (data, twoPpath ,sessionName, True, figsavepath, nargout=2)
                    info.recordingList.loc[ind,'variance'] = variance

                    # Apply correction to the signal
                    data = dataCorrected
                    
                # Save the file
                eng.writetable(data, filenameCSV, nargout=0)
                info.recordingList.loc[ind,'eventTimesExtracted']=1
            except:
                print(str(ind) + ' - FAILED: Extracting time events: ' + info.recordingList.sessionName[ind])
                info.recordingList.loc[ind,'eventTimesExtracted']=0

# display the output
print( "Behaviour trial data extraction completed: " + 
      str(info.recordingList.eventTimesExtracted.sum()) +"/" + str(info.recordingList.shape[0]))

 Extracting time events--> Profile: Value2AFC_noTimeline  Session: 2023-05-31_1_OFZ011
0 - FAILED: Extracting time events: 2023-05-31_1_OFZ011
Behaviour trial data extraction completed: 0.0/1


In [18]:
len(filenameTimeline)

0

In [ ]:
# Check Suite2p files
# Run: suite2p_run.py to create files in suite2p enviroment. 
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    filenameDFF = info.recordingList.analysispathname[ind] + 'imaging-data.pkl'
    e_filenameDFF = [f for f in glob.glob(filenameDFF)]
    if len(e_filenameDFF)>0:
        info.recordingList.loc[ind,'imagingDataExtracted']=1
        info.recordingList.loc[ind,'imagingDataPath'] = filenameDFF
    else:
        info.recordingList.loc[ind,'imagingDataExtracted']=0
        info.recordingList.loc[ind,'imagingDataPath'] = filenameDFF

# display the output
print( "Imaging data extraction completed: " + 
      str(info.recordingList.imagingDataExtracted.sum()) +"/" + str(info.recordingList.shape[0]))

In [ ]:
# Save info into the analysis folder
filenameINFO = info.analysisPath + '\\infoForAnalysis.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

# Save table as CSV
recordingList = info.recordingList
recordingList.to_csv( info.analysisPath +'\\recordingList.csv', index=False)